Solution 2: using Transfer learning to boost the Accuracy 

Val_accuracy: 100% - test accuracy: 98.8%

parameters:
pre-tianed model: InceptionResNetV2
777 conv layaers 
3 Dense layers - relu activation  
1 softmax 

optimizer: ADAM with lr = 0.002
dropout: 50% 
Augmentation: yes
class weights added to compensate for data imbalance

note: i have a problem with result reproducibility, I have fixed a seed for tensorflow, keras and used keras intitilaiztion in the dense layers. but the results differ a little bit for run to another. I think this randomness comes from cloud GPU. 

Impoting data from googe drive and importing the useful libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
base_dir = "/content/drive/My Drive/classification/classification/Classification_Dataset/training"

In [3]:
#try:
  # Use the %tensorflow_version magic if in colab.
  #%tensorflow_version 2.x
#except Exception:
  #pass

import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
import os
import cv2
import glob
import shutil
import numpy as np
import keras
from keras.applications.inception_v3 import InceptionV3
from keras import backend as K
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.models import Model
from keras.layers import Dense,GlobalAveragePooling2D,Dropout,SeparableConv2D, BatchNormalization, Activation,  Conv2D, Flatten, MaxPooling2D
from keras.applications.mobilenet import MobileNet
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.datasets import load_files  # to load the images from the subfolders    
from keras.utils import np_utils # to hot encode the label of the image
import matplotlib.pyplot as plt # to draw barplot and etc...
from tqdm import tqdm 
from keras.preprocessing import image    
from keras.callbacks import ModelCheckpoint # to save the best weights for the model while training 
import random as rn
from sklearn.utils import class_weight
from datetime import datetime
import cv2
from PIL import Image



Using TensorFlow backend.


Fixing a random seed to make the results reproducible

In [0]:
rand_seed = 77
os.environ['PYTHONHASHSEED'] = '0'

#random seed for NP genreator of ranodm numbers
np.random.seed(rand_seed)

#random seed generator for Python
rn.seed(rand_seed)

#random seed for tensorflow
tf.set_random_seed(rand_seed)

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

defining the classes in the correct order {owl: 0, galaxy: 1, ... 

In [0]:
class_list = [
              'owl',
              'galaxy',
              'lightning',
              'wine-bottle',
              't-shirt',
              'waterfall',
              'sword',
              'school-bus',
              'calculator',
              'sheet-music',
              'airplanes',
              'lightbulb',
              'skyscraper',
              'mountain-bike',
              'fireworks',
              'computer-monitor',
              'bear',
              'grand-piano',
              'kangaroo',
              'laptop'
              ]

 since the data is not splitted into training and vailation sets. i will do so in the next code .The code below creates a train and a val folder each containing 20 folders (one for each type of class). It then moves the images from the original folders to these new folders such that 80% of the images go to the training set and 20% of the images go into the validation set. In the end our directory will have the 

Since I did nott delete the original folders, they will still be in the directory which is ok

In [0]:
train_list = []
val_list = []
train_dict = {}
val_dict = {}

for cl in class_list:
  img_path = os.path.join(base_dir, cl)
  #print(img_path)
  images = glob.glob(img_path + '/*')
  #print("{}: {} Images".format(cl, len(images)))
  num_train = int(round(len(images)*0.8))
  train, val = images[:num_train], images[num_train:]

  for t in train:
    if not os.path.exists(os.path.join(base_dir, 'train', cl)):
      os.makedirs(os.path.join(base_dir, 'train', cl))
    shutil.move(t, os.path.join(base_dir, 'train', cl))

  for v in val:
    if not os.path.exists(os.path.join(base_dir, 'val', cl)):
      os.makedirs(os.path.join(base_dir, 'val', cl))
    shutil.move(v, os.path.join(base_dir, 'val', cl))
############################################################### Generating JSON file for data Split ##########################################
  for image in glob.glob(os.path.join(base_dir, 'train', cl) + '/*.jpg'):
      train_list.append(image)
      train_dict.setdefault (cl , []).append(os.path.basename(image)) 
      #print(image)
      
  for image in glob.glob(os.path.join(base_dir, 'val', cl) + '/*.jpg'):
      val_list.append(image)
      val_dict.setdefault (cl , []).append(os.path.basename(image))   
   
#dataset_split = {'training' : train_dict , 'validation' : val_dict }

#print(dataset_split)
#import json
#with open('dataset_split.json', 'w') as fp:
#      json.dump(dataset_split, fp)


In [0]:
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')

In the cell below, ImageDataGenerator transform and rescale the images by 255 and that applies:

- random 45 degree rotation
- random zoom of up to 50%
- random horizontal flip
- random vertical flip

flow_from_directory method to apply the above transformation to the images in the training set. batch size, the path to the directory of the training images, the target size for the images, to shuffle the images, the class mode is set to sparse

same applies to validation side but without augmnetation nor shuffling

In [9]:
batch_size = 64
IMG_SHAPE = 299

image_gen_train = ImageDataGenerator(
                   rescale=1./255,               
                   vertical_flip = True,
                   horizontal_flip = True,
                   rotation_range=45,
                   zoom_range=0.5,                  
                   )

train_data_gen = image_gen_train.flow_from_directory(
                                                batch_size=32,
                                                directory=train_dir,
                                                classes = class_list,
                                                shuffle=True,
                                                target_size=(IMG_SHAPE,IMG_SHAPE),
                                                class_mode='sparse',
                                                seed = rand_seed                                                
                                                )


image_gen_val = ImageDataGenerator(rescale=1./255)
val_data_gen = image_gen_val.flow_from_directory(
                                                 batch_size=16,
                                                 directory=val_dir,
                                                 classes = class_list,
                                                 shuffle=True,
                                                 target_size=(IMG_SHAPE, IMG_SHAPE),
                                                 class_mode='sparse',
                                                 seed = rand_seed
                                                 )

Found 1245 images belonging to 20 classes.
Found 309 images belonging to 20 classes.


convolutional neural network that consists of pre-trained model on Image net data set. the model is InceptionResNetV2.

After the model is followed by 4 dense layers followed by a fully connected layer with 1024,512,256,20 units each. The CNN outputs class probabilities based on 20 classes which is done by the softmax activation function. All other layers use a relu activation function. add Dropout layers 0.5 probability is added

In [10]:


# Base model without Fully connected Layers
base_model = keras.applications.inception_resnet_v2.InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(299,299,3), pooling=max)
x=base_model.output

# Add some new Fully connected layers
x=GlobalAveragePooling2D()(x)

x=Dense(1024,kernel_initializer=keras.initializers.glorot_uniform(seed=rand_seed),activation='relu')(x)
x = Dropout(0.5)(x)

x=Dense(512,kernel_initializer=keras.initializers.glorot_uniform(seed=rand_seed),activation='relu')(x) 
x = Dropout(0.5)(x)

x=Dense(256,kernel_initializer=keras.initializers.glorot_uniform(seed=rand_seed),activation='relu')(x) 
x = Dropout(0.5)(x)

preds=Dense(20,kernel_initializer=keras.initializers.glorot_uniform(seed=rand_seed),activation='softmax')(x) #final layer with softmax activation

model=Model(inputs=base_model.input,outputs=preds)











Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [11]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [12]:
for i,layer in enumerate(model.layers):
  print("{}: {}".format(i,layer))

0: <keras.engine.input_layer.InputLayer object at 0x7f5903c6ab00>
1: <keras.layers.convolutional.Conv2D object at 0x7f5903c6add8>
2: <keras.layers.normalization.BatchNormalization object at 0x7f5903c6aa58>
3: <keras.layers.core.Activation object at 0x7f58415fce48>
4: <keras.layers.convolutional.Conv2D object at 0x7f58415fcfd0>
5: <keras.layers.normalization.BatchNormalization object at 0x7f5841de5ba8>
6: <keras.layers.core.Activation object at 0x7f5841e02e80>
7: <keras.layers.convolutional.Conv2D object at 0x7f5841e02eb8>
8: <keras.layers.normalization.BatchNormalization object at 0x7f5841dc4358>
9: <keras.layers.core.Activation object at 0x7f5841dc4898>
10: <keras.layers.pooling.MaxPooling2D object at 0x7f5841dcbcc0>
11: <keras.layers.convolutional.Conv2D object at 0x7f5841dddb70>
12: <keras.layers.normalization.BatchNormalization object at 0x7f5841d8a940>
13: <keras.layers.core.Activation object at 0x7f5841d8af60>
14: <keras.layers.convolutional.Conv2D object at 0x7f5841d8af98>
15: <

In [0]:
for layer in model.layers[:777]:
    layer.trainable=False
for layer in model.layers[777:]:
    layer.trainable=True

class wieghts are used to deal with the imbalance in the data set. later on these realtive weights are inputted to the model.fit_generator.


In [0]:
class_weights = {
		0	:	1.052631579	,
		1	:	1.538461538	,
		2	:	1	,
		3	:	1.315789474	,
		4	:	1	,
		5	:	1.428571429	,
		6	:	1.298701299	,
		7	:	1.369863014	,
		8	:	1.333333333	,
		9	:	1.694915254	,
		10	:	1	,
		11	:	1.492537313	,
		12	:	1.428571429	,
		13	:	1.754385965	,
		14	:	1.333333333	,
		15	:	1	,
		16	:	1.298701299	,
		17	:	1.428571429	,
		18	:	1.754385965	,
		19	:	1	
}

The cell below compiles the model. ADAM and SGD optimizers are added for experimenting, the sparse cross entropy function as a loss functn.training and validation accuracy on each epoch 

In [15]:
epochs = 100
learning_rate = 0.0025
decay_rate = learning_rate / epochs

opt1 = SGD(lr=learning_rate,
            momentum = .08, 
            decay = decay_rate
           )

opt2 = Adam(lr=learning_rate, 
            beta_1=0.9, beta_2=0.999, epsilon=None, 
            decay=decay_rate, amsgrad=False)

model.compile(optimizer=opt2,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

The cell below trains model using the fit_generator function. checkpoint are included to capture the best validtion_loss weights and class wieghts take care of the imbalance in the data set  

In [1]:
checkpointer = ModelCheckpoint(filepath='/content/drive/My Drive/weights.best.InceptionResNetV2_2.hdf5', 
                               monitor='val_loss',
                               verbose=1, 
                               save_best_only=True
                               )

history = model.fit_generator(
                              train_data_gen,
                              steps_per_epoch=int(np.ceil(train_data_gen.n / float(batch_size))),
                              epochs=100,
                              validation_data=val_data_gen,
                              validation_steps=int(np.ceil(val_data_gen.n / float(batch_size))),
                              callbacks = [checkpointer],
                              class_weight = class_weights,
                              verbose=1
                              )

NameError: ignored

visulizing the accuracies and losses of both training and validation

In [0]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

The remaining cells do proccesing to convert images to 4D, to predicat on the test data and to generate CSV submission for Kaggle. at the end the predicted image is plotted along with the respective model prediction  

In [0]:
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(299, 299))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 299, 299, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [0]:
model.load_weights('/content/drive/My Drive/weights.best.InceptionResNetV2_2.hdf5')

In [0]:
def predict_img(img_path):
    img=path_to_tensor(img_path).astype('float32')/255
    # obtain predicted vector
    predicted_vector = model.predict(img)
    # return dog breed that is predicted by the model
    return [np.argmax(predicted_vector)]

In [0]:
from keras.preprocessing import image
type(predict_img('/content/drive/My Drive/classification (1)/Classification_Dataset/test/IMG_1035.jpg'))

In [0]:
def conversion2kaggle (prediction):
    return {
        '[12]'	:	12,
        '[5]'	  :	5,
        '[10]'	:	10,
        '[19]'	:	19,
        '[17]' 	:	17,
        '[18]'  :	18,
        '[16]'  :	16,
        '[13]'  :	13,
        '[2]'	  :	2,
        '[14]'	:	14,
        '[0]'	  :	0,
        '[9]'	  :	9,
        '[15]' 	:	15,
        '[11]'	:	11,
        '[4]'	  :	4,
        '[3]'	  :	3,
        '[1]' 	:	1,
        '[6]'	  :	6,
        '[7]'   :	7,
        '[8]'  	:	8,
    }[prediction]

In [0]:
test = load_files("/content/drive/My Drive/classification (1)/Classification_Dataset/test") #this line takes too much time on colab to execute 5 min 

In [0]:
def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

In [0]:
image_list = []
results = {}

for filename in glob.glob('/content/drive/My Drive/classification (1)/Classification_Dataset/test/*.jpg'): 
    image_list.append(filename)


for image_name in image_list:
    results[os.path.basename(image_name)]=conversion2kaggle((str(predict_img(image_name))))

print(results)
create_csv(results)

In [0]:
def show_img(img_path): 
  img = cv2.imread(img_path)
  cv_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  plt.imshow(cv_rgb)
  plt.show()
  print("     Predicted as {}\n\n\n".format(class_list[conversion2kaggle((str(predict_img(img_path))))])
   
)

In [0]:
for path in image_list:
  show_img(path)